In [1]:
%conda install  matplotlib=3.0.3

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\akeshava\.conda\envs\conda_env

  added / updated specs:
    - matplotlib=3.0.3


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.27 |                0         163 KB
    certifi-2019.11.28         |           py37_0         157 KB
    ------------------------------------------------------------
                                           Total:         320 KB

The following packages will be UPDATED:

  ca-certificates                              2019.10.16-0 --> 2019.11.27-0
  certifi                                  2019.9.11-py37_0 --> 2019.11.28-py37_0




ca-certificates-2019 | 163 KB    |            |   0% 
ca-certificates-2019 | 163 KB    | 9          |  10% 
ca-certificates-2019 | 163 KB    | ########## | 100% 

certifi-2019.11.28   | 157 KB    |            |   0%



==> WARNING: A newer version of conda exists. <==
  current version: 4.7.10
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda




In [4]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.io as spio
import numpy as np
# from scipy.misc import imresize
from scipy.spatial import distance
from IPython.display import display
from skimage.transform import rescale, resize
import scipy.stats as spstat
import statsmodels.formula.api as smf
from statsmodels.regression.linear_model import WLS
from patsy import dmatrix, dmatrices, ContrastMatrix, Poly
# OG_DATA_PATH = './TaskPerformance/'
PLOTS_PATH = './Seahaven_VR_Map_comp_paper/'
POS_PATH = './position/'
CSV_PATH = './csv_files'
EYE_DATA_PATH = './ET_data/'
CSV_PATH = './csv_files'
MOUSE_DATA_PATH = './MouseData'
os.makedirs(os.path.dirname(PLOTS_PATH), exist_ok=True)


# 1. Accuracy as a function of group, task and time

In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_group_task_time_accuracy.csv')
display(tmpdf.info())
tmpdf.Accuracy = tmpdf.Accuracy*100
tmpdf = (
    tmpdf
    .groupby(['Group','Subject','Task','Time'])
    .agg({'Accuracy':'mean'})
    .rename(columns={'Accuracy':'Accuracy [%]'})
    .reset_index()
#     .query('Time == "Inf"')
)

display(tmpdf)
sns.set(context = "paper", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(12,9)})
g = sns.catplot(data=tmpdf, x='Group', y='Accuracy [%]',hue='Time', kind='point',
                col='Task', col_order=['Absolute','Relative','Pointing'], order=["VR", "Map"],
                 ci=68, capsize=0.05, 
                dodge=0.1,scale=2
#                  edgecolor=(0,0,0),errwidth=3,linewidth=2.5
           )

g.set(yticks=np.arange(40,80,10))
i=0
for ax,title in zip(g.axes.flat,['Absolute','Relative','Pointing']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i != 1 :
        ax.spines['left'].set_visible(False)
    ax.set(ylim=(40,70))
    ax.set_title(title)
plt.savefig(PLOTS_PATH+'/group_task_time_accuracy.png', quality=90)
plt.savefig(PLOTS_PATH+'/group_task_time_accuracy.eps', quality=90)

# 2. Accuracy as function of Alignment to North

In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_group_angletonorth_time_accuracy.csv')
display(tmpdf.info())
tmpdf.Accuracy = tmpdf.Accuracy*100
tmpdf = (
    tmpdf
    .groupby(['Group','Subject','AngleToNorth','Time'])
    .agg({'Accuracy':'mean'})
    .rename(columns={'Accuracy':'Accuracy [%]'})
    .reset_index()
#     .query('Time == "Inf"')
)

display(tmpdf)
sns.set(context = "paper", style="white", palette="Set1", font_scale=2, rc={'figure.figsize':(12,15)})
g = sns.catplot(data=tmpdf, x='AngleToNorth', y='Accuracy [%]',hue='Time', 
                kind='point',dodge=0.2,scale = 1.5,
                col='Group', 
                ci=68, capsize=0.1, 
#                  edgecolor=(0,0,0),errwidth=3,linewidth=2
           )

g.set(yticks=np.arange(30,90,10))
i=0
for ax,title in zip(g.axes.flat,['VR','Map']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i != 1 :
        ax.spines['left'].set_visible(False)
    ax.set(ylim=(30,80),xlabel='Alignment To North [$^\circ$]')
    ax.set_title(title)
    
plt.savefig(PLOTS_PATH+'/group_alignmentToNorth_time_accuracy.png', quality=90)

# 3. Accuracy as function of angular difference between houses

In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_group_angulardifference_time_accuracy.csv')
display(tmpdf.info())
tmpdf.Accuracy = tmpdf.Accuracy*100
tmpdf = (
    tmpdf
    .groupby(['Group','Subject','AngularDiff','Time'])
    .agg({'Accuracy':'mean'})
    .rename(columns={'Accuracy':'Accuracy [%]'})
    .reset_index()
#     .query('Time == "Inf"')
)

display(tmpdf)
sns.set(context = "paper", style="white", palette="Set1", font_scale=2, rc={'figure.figsize':(12,15)})
g = sns.catplot(data=tmpdf, x='AngularDiff', y='Accuracy [%]',hue='Time', 
                kind='point',dodge=0.2,scale = 1.5,
                col='Group', 
                ci=68, capsize=0.1, 
#                  edgecolor=(0,0,0),errwidth=3,linewidth=2
           )

g.set(yticks=np.arange(30,90,10))
i=0
for ax,title in zip(g.axes.flat,['VR','Map']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i != 1 :
        ax.spines['left'].set_visible(False)
    ax.set(ylim=(40,70),xlabel='Angular Difference [$^\circ$]')
    ax.set_title(title)
    
plt.savefig(PLOTS_PATH+'/group_angulardifference_time_accuracy.png', quality=90)

# 4. Accuracy as a function of distance

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_VR_rel_point_distance_accuracy.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_map_rel_point_distance_accuracy.csv')]
)
tmpdf.Accuracy = tmpdf.Accuracy*100
# display(tmpdf)

sns.set(context = "paper", style="white", palette="Set1", font_scale=3, rc={'figure.figsize':(10,10)})

g = sns.lmplot(data=tmpdf, x='distance_prime_target', y='Accuracy',hue='Time', 
               col='Task',row='Group', 
               x_ci='ci', ci=95, n_boot=1000,robust=True,
               height=8, aspect=1.2, 
               scatter_kws={"s": 200, 'edgecolors':'k', 'linewidths': 3},
               line_kws={'linewidth': 5})

i=0
for ax,title in zip(g.axes.flat,['VR - Relative Task','VR - Pointing Task','Map - Relative Task', 'Map - Pointing Task']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i != 1 and i!=3 :
        ax.spines['left'].set_visible(False)
#     ax.set(ylim=(40,70),xlabel='Angular Difference [$^\circ$]')
    ax.set_title(title)

g = (
    g
    .set_axis_labels("Distance [m]", "Accuracy[%]")
    .set(yticks=[0,20,40,60,80,100], ylim=(0,100))
    .set(xticks=[100,200,300,400], xlim=(0,400))
)

plt.savefig(PLOTS_PATH+'/group_distance_time_accuracy.png', quality=90)

# 4.1 Modelling Accuracy as function of distance for Relative Task

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_VR_rel_point_distance_accuracy.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_map_rel_point_distance_accuracy.csv')]
)
df = tmpdf.query('Task=="Relative"')
df.Accuracy = df.Accuracy*100

model = smf.ols('Accuracy ~ C(Group,Diff)+C(Time,Diff) + distance_prime_target +\
                C(Time,Diff):distance_prime_target + C(Group,Diff):distance_prime_target + \
                C(Group,Diff):C(Time,Diff):distance_prime_target',df).fit()
print('GROUP RELATIVE distance effect complete model')
display(model.summary())


# display(np.asarray(dmatrix("C(Group, Treatment(reference='Map'))", df)))
# display(np.asarray(dmatrix('C(Group,Diff)', df)))

# 4.2 Modelling Accuracy as a function of DIstance for Pointing Task

In [21]:
df = tmpdf.query('Task=="Pointing"')
df.Accuracy = df.Accuracy*100

model = smf.ols('Accuracy ~ C(Group,Diff)+C(Time,Diff) + distance_prime_target +\
                C(Time,Diff):distance_prime_target + C(Group,Diff):distance_prime_target + \
                C(Group,Diff):C(Time,Diff):distance_prime_target',df).fit()
print('GROUP Pointing distance effect complete model')
display(model.summary())


# display(np.asarray(dmatrix("C(Group, Treatment(reference='Map'))", df)))
# display(np.asarray(dmatrix('C(Group,Diff)', df)))

GROUP RELATIVE distance effect complete model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Accuracy   R-squared:                       0.178
Model:                            OLS   Adj. R-squared:                  0.142
Method:                 Least Squares   F-statistic:                     4.929
Date:                Wed, 04 Dec 2019   Prob (F-statistic):           0.000135
Time:                        18:05:08   Log-Likelihood:                -562.09
No. Observations:                 144   AIC:                             1138.
Df Residuals:                     137   BIC:                             1159.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          61.3816      2.238     27.427      0.000      56.956      65.807
C(Group, Diff)[D.Map]                                              -9.4984      4.476     -2.122      0.036     -18.349      -0.648
C(Time, Diff)[D.3s]                                                -5.5365      4.476     -1.237      0.218     -14.387       3.314
distance_prime_target                                              -0.0423      0.015     -2.900      0.004      -0.071      -0.013
C(Time, Diff)[D.3s]:distance_prime_target                           0.0756      0.029      2.593      0.011       0.018       0.133
C(Group, Diff)[D.Map]:distance_prime_target                         0.0137      0.029      0.474      0.636      -0.043       0.071
C(Group, Diff)[D.Map]:C(Time, Diff)[D.3s]:distance_prime_target     0.0362      0.028      1.307      0.193      -0.019       0.091
==============================================================================
Omnibus:                        3.715   Durbin-Watson:                   1.990
Prob(Omnibus):                  0.156   Jarque-Bera (JB):                3.358
Skew:                          -0.369   Prob(JB):                        0.187
Kurtosis:                       3.117   Cond. No.                         710.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

GROUP Pointing distance effect complete model


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Accuracy   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.065
Method:                 Least Squares   F-statistic:                     2.659
Date:                Wed, 04 Dec 2019   Prob (F-statistic):             0.0180
Time:                        18:05:08   Log-Likelihood:                -590.62
No. Observations:                 144   AIC:                             1195.
Df Residuals:                     137   BIC:                             1216.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================================================================
                                                                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          53.0155      2.543     20.850      0.000      47.988      58.043
C(Group, Diff)[D.Map]                                               3.3571      5.060      0.663      0.508      -6.648      13.362
C(Time, Diff)[D.3s]                                                 8.2523      5.085      1.623      0.107      -1.804      18.308
distance_prime_target                                               0.0101      0.014      0.745      0.457      -0.017       0.037
C(Time, Diff)[D.3s]:distance_prime_target                           0.0072      0.027      0.267      0.790      -0.046       0.061
C(Group, Diff)[D.Map]:distance_prime_target                         0.0010      0.027      0.038      0.970      -0.052       0.055
C(Group, Diff)[D.Map]:C(Time, Diff)[D.3s]:distance_prime_target     0.0137      0.027      0.505      0.614      -0.040       0.067
==============================================================================
Omnibus:                        6.393   Durbin-Watson:                   2.274
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                6.272
Skew:                           0.465   Prob(JB):                       0.0435
Kurtosis:                       2.577   Cond. No.                         775.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# 4.2 Modelling Accuracy as function of distance for Pointing Task

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_VR_rel_point_distance_accuracy.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_map_rel_point_distance_accuracy.csv')]
)
df = tmpdf.query('Task=="Pointing"')
display(df.groupby(['Group','Time','distance_prime_target']).agg('count'))
model = smf.ols('Accuracy ~ C(Group)+C(Time) + distance_prime_target +\
                C(Time)*distance_prime_target + C(Group)*distance_prime_target + \
                C(Group)*C(Time)*distance_prime_target',df).fit()
print('GROUP Pointing distance effect complete model')
display(model.summary())
df = tmpdf.query('Group =="VR" and Task=="Pointing" and Time=="3s"')
model = smf.ols('Accuracy ~ distance_prime_target',df).fit()
print('Distance effect for VR Pointing 3s')
display(model.summary())
df = tmpdf.query('Group =="VR" and Task=="Pointing" and Time=="Inf"')
model = smf.ols('Accuracy ~ distance_prime_target',df).fit()
print('Distance effect for VR Pointing Inf')
display(model.summary())
df = tmpdf.query('Group =="map" and Task=="Pointing" and Time=="3s"')
model = smf.ols('Accuracy ~ distance_prime_target',df).fit()
print('Distance effect for Map Pointing 3s')
display(model.summary())
df = tmpdf.query('Group =="map" and Task=="Pointing" and Time=="Inf"')
model = smf.ols('Accuracy ~ distance_prime_target',df).fit()
print('Distance effect for Map Pointing Inf')
display(model.summary())

# 5.1 Accuracy as function of FRS scales in Map group

In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_map_task_time_FRS_long.csv')
tmpdf.Accuracy = tmpdf.Accuracy*100
# display(tmpdf.head())
sns.set(context = "paper", style="white", palette="Set1", font_scale=3, rc={'figure.figsize':(30,10)})

tmpdf = (tmpdf
         .query('Scale == "Cardinal Direction"')
         .rename(columns={'Accuracy': 'Accuracy [%]'})
)
g = sns.lmplot(data=tmpdf, x='Score', y='Accuracy [%]',hue='Time', 
               col='Task', row='Scale',
               x_ci='ci', ci=95, n_boot=1000,
#                robust=True,
               height=8, aspect=1.2, 
               sharex=False, 
               scatter_kws={"s": 200, 'edgecolors':'k', 'linewidths': 3},
               line_kws={'linewidth': 5})

i=0
for ax,title in zip(g.axes.flat,['Absolute','Relative','Pointing']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i not in [1,4,7]:
        ax.spines['left'].set_visible(False)
        
    ax.set(ylim=(0,100),xlim=(0,7), xlabel = 'Cardinal Direction Score')
    ax.set_title(title)
# plt.show()
plt.savefig(PLOTS_PATH+'/Map_task_time_FRS_correlation.png', quality=90)

# 5.2  Accuracy as function of FRS scales in VR group


In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_VR_task_time_FRS_long.csv')
tmpdf.Accuracy = tmpdf.Accuracy*100

sns.set(context = "paper", style="white", palette="Set1", font_scale=3, rc={'figure.figsize':(30,10)})

tmpdf = (tmpdf
         .query('Scale == "Cardinal Direction"')
         .rename(columns={'Accuracy': 'Accuracy [%]'})
)
g = sns.lmplot(data=tmpdf, x='Score', y='Accuracy [%]',hue='Time', 
               col='Task', row='Scale',
               x_ci='ci', ci=95, n_boot=1000,
#                robust=True,
               height=8, aspect=1.2, 
               sharex=False, 
               scatter_kws={"s": 200, 'edgecolors':'k', 'linewidths': 3},
               line_kws={'linewidth': 5})

i=0
for ax,title in zip(g.axes.flat,['Absolute','Relative','Pointing']):
    i = i+1
    ax.axhline(50, ls=':', color='k', lw=4)
    if i not in [1,4,7]:
        ax.spines['left'].set_visible(False)
        
    ax.set(ylim=(0,100),xlim=(0,7), xlabel = 'Cardinal Direction Score')
    ax.set_title(title)
# plt.show()
plt.savefig(PLOTS_PATH+'/VR_task_time_FRS_correlation.png', quality=90)

# 5.3 Modelling accuracy vs FRS scales 
### 5.3.1 Both Groups combined

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_map_task_time_FRS_long.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_VR_task_time_FRS_long.csv')]
)
df = tmpdf.query('Task == "Absolute"')
model = smf.ols('Accuracy ~ C(Group) + C(Time) + C(Scale)',df).fit()
print('Absolute Task FRS model')
display(model.summary())
df = tmpdf.query('Task == "Relative"')
model = smf.ols('Accuracy ~ C(Group) + C(Time) + C(Scale) ',df).fit()
print('Relative Task FRS model')
display(model.summary())
df = tmpdf.query('Task == "Pointing"')
model = smf.ols('Accuracy ~ C(Group) + C(Time) + C(Scale)',df).fit()
print('Pointing Task FRS model')
display(model.summary())

### 5.3.2 Map group,  Task, Time, Scale separated

** Both OLS and spearmanr correlation are performed. Spearmanr correlation should be considered since our data is not normally distributed. OLS will give wrong results

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_map_task_time_FRS_long.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_VR_task_time_FRS_long.csv')]
)
df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="3s" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Absolute Task 3s FRS Egocentric model')
display(model.summary()) # displays the OLS model
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit')) # displays the correlation + pvalue
df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="3s" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Absolute Task 3s FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="3s" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Absolute Task 3s FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))



df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="Inf" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Absolute Task Inf FRS Egocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="Inf" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ C(Scale)-1',df).fit()
print('Absolute Task Inf FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Absolute" and Time=="Inf" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Absolute Task Inf FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))


df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="3s" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task 3s FRS Egocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="3s" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task 3s FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="3s" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task 3s FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))



df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="Inf" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task Inf FRS Egocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="Inf" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task Inf FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Relative" and Time=="Inf" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Relative Task Inf FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))


df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="3s" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task 3s FRS Egocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="3s" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task 3s FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="3s" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task 3s FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))



df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="Inf" and Scale =="Egocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task Inf FRS Egocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="Inf" and Scale =="Allocentric" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task Inf FRS Allocentric model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="Map" and Task == "Pointing" and Time=="Inf" and Scale =="Cardinal Direction" ')
model = smf.ols('Accuracy ~ Score -1',df).fit()
print('Pointing Task Inf FRS Cardinal Direction model')
display(model.summary())
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

### 5.3.2 VR group,  Task, Time, Scale separated
** Only showing Spearmanr correlation results

In [ ]:
tmpdf = pd.concat(
    [pd.read_csv(CSV_PATH+'/seahaven_map_task_time_FRS_long.csv'),
     pd.read_csv(CSV_PATH+'/seahaven_VR_task_time_FRS_long.csv')]
)
df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="3s" and Scale =="Egocentric" ')
print('Absolute Task 3s FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="3s" and Scale =="Allocentric" ')
print('Absolute Task 3s FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="3s" and Scale =="Cardinal Direction" ')
print('Absolute Task 3s FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))


df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="Inf" and Scale =="Egocentric" ')
print('Absolute Task Inf FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="Inf" and Scale =="Allocentric" ')
print('Absolute Task Inf FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Absolute" and Time=="Inf" and Scale =="Cardinal Direction" ')
print('Absolute Task Inf FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))


df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="3s" and Scale =="Egocentric" ')
print('Relative Task 3s FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="3s" and Scale =="Allocentric" ')
print('Relative Task 3s FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="3s" and Scale =="Cardinal Direction" ')
print('Relative Task 3s FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))



df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="Inf" and Scale =="Egocentric" ')
print('Relative Task Inf FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="Inf" and Scale =="Allocentric" ')
print('Relative Task Inf FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Relative" and Time=="Inf" and Scale =="Cardinal Direction" ')
print('Relative Task Inf FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))


df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="3s" and Scale =="Egocentric" ')
print('Pointing Task 3s FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="3s" and Scale =="Allocentric" ')
print('Pointing Task 3s FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="3s" and Scale =="Cardinal Direction" ')
print('Pointing Task 3s FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))



df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="Inf" and Scale =="Egocentric" ')
print('Pointing Task Inf FRS Egocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="Inf" and Scale =="Allocentric" ')
print('Pointing Task Inf FRS Allocentric ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))

df = tmpdf.query('Group =="VR" and Task == "Pointing" and Time=="Inf" and Scale =="Cardinal Direction" ')
print('Pointing Task Inf FRS Cardinal Direction ')
display(spstat.spearmanr(df.Accuracy.values, df.Score.values, nan_policy='omit'))




# 7. Reading path data of valid subjects and drawing walked path

In [ ]:
subdf = pd.read_csv('./Seahaven_alignment_project_24092019.csv', sep=';')
tmpdf = (
    subdf
    .query('Discarded != "yes"')
    .query('Measurement == 3')
    .query('Training == "VR"')
    [['Subject','Comments']] 
)
tmpdf[['m1','m2']] = tmpdf.Comments.str.replace(r'[A-Za-z\s\-\&\.\']','').str.strip().str.extract(r'\#([0-9]*)\#([0-9]*)')
subs = tmpdf.Subject.to_list()
subs.extend(tmpdf.m1)
subs.extend(tmpdf.m2)
display(len(subs))
columnnames = ['x', 'z', 'y', 'rx', 'ry', 'rz', 'timestamp', 'PL_timestamp' ]
pathdf = pd.DataFrame()
for subj in subs:
    tmpdf = pd.read_csv(POS_PATH + f'positions_VP{subj}.txt', header=None, names=columnnames)
    tmpdf['subject'] = subj
    pathdf = pd.concat([pathdf, tmpdf], ignore_index=True)

pathdf.x = pathdf.x.astype(int) - 180
pathdf.y = pathdf.y.astype(int) - 535
    


In [ ]:
from matplotlib.colors import LogNorm, PowerNorm
map_img = plt.imread('./map_grey_1.png')
# map_img = scipy.misc.imresize(map_img,size=0.5)
# H, xe, ye = np.histogram2d(pathdf.x.values, pathdf.y.values, bins=100)
# X,Y = np.meshgrid(xe,ye)
# display(H.shape)
sns.set(context = "poster", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(12,15)})

_, ax = plt.subplots(figsize=(20,20))

im = ax.imshow(map_img, extent=[1, 450, 500,1], origin = 'lower')
# h, xe, ye, img = ax.hist2d(pathdf.y.values,pathdf.x.values, bins=[143,161],cmap='Blues', norm=LogNorm())
H, xe, ye = np.histogram2d(pathdf.x.values, pathdf.y.values, bins=[143,161])
# img = ax.imshow(H, interpolation='nearest', origin='low',extent=[xe[0], xe[-1], ye[0], ye[-1]])
extent = [xe.min(),xe.max(),ye.min(),ye.max()]
img = ax.imshow(np.ma.masked_where(H == 0,H), interpolation='none', extent=[1, 450, 500,1], origin='lower', 
                cmap='Blues', norm=LogNorm())
# display(h[0].shape)
cbar = plt.colorbar(img, orientation= 'vertical' )

# cbar.set_label('Density', rotation=270)
# cbar.ax.set_yticklabels(['0', '10', '66', '100'], fontsize=30)
# cbar.ax.set_yticks(['0', '10', '66', '100'], fontsize=30)
# cbar.ax.set_yscale('Log')

plt.gca().invert_yaxis()
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
# plt.show()
plt.savefig(PLOTS_PATH+'/walked_paths.png', quality=90)

# 8. Familiarity of houses based on dwelling time (VR)

In [ ]:
rep_measurements = pd.read_csv('{}/combinedSessions_newPartNumbers.csv'.format(EYE_DATA_PATH))
house_df = pd.read_csv('{}/seahaven_house_info.csv'.format(CSV_PATH))
display(house_df.info())
display(rep_measurements.head())

In [ ]:
def mad_based_outlier(points, thresh=3.5):
    """ Median Absolute deviation based outlier detection.
    Ideally this method should be used to clean fixation durations."""
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score < thresh

In [ ]:
eye_df = pd.DataFrame()
for subID in rep_measurements.newPartNumber.values:
    try:
        tmpdf = pd.read_csv('{}gazes_{}.csv'.format(EYE_DATA_PATH,subID))
        tmpdf['Subject'] = subID
        eye_df = pd.concat([eye_df, tmpdf], ignore_index=True)
    except OSError:
        print('{}gazes_{}.csv not found, Moving on.'.format(EYE_DATA_PATH,subID))   

display(eye_df.describe())

eye_df = eye_df.query('House != "NH" and House != "sky" and House != "noData"')
print("House with maximum dwelling time:")
display(eye_df.loc[eye_df.Time.argmax()])
ax = eye_df.Time.hist(bins=50, label='before filtering', alpha=0.5,)
# dwelling time has large outliers, needs to be cleaned
eye_df = eye_df.loc[mad_based_outlier(eye_df.Time)]
display(eye_df.describe)
eye_df.Time.hist(bins=50, label='after filtering',alpha=0.5, ax=ax)

In [ ]:
eye_df.House = eye_df.House.str.strip()
# sub_fixDur = eye_df.groupby(['Subject','House']).agg({'Time':'mean'})
eye_df[['HouseNr', 'Angle', 'Other']] = eye_df.House.str.split('_', expand=True)
eye_df.HouseNr = eye_df.HouseNr.astype(int)
eye_df = eye_df.merge(house_df, on = 'HouseNr')
sub_fixDur = eye_df.groupby(['HouseNr','x', 'y' ]).agg({'Time':'mean'}).reset_index()
display(f'Mean dwelling Time over houses and subjects: {sub_fixDur.Time.mean()}ms (SD={sub_fixDur.Time.std()})')
# rescaling x,y coordinates to fit 2D Map
sub_fixDur.x = sub_fixDur.x.astype(int) - 180
sub_fixDur.y = sub_fixDur.y.astype(int) - 535
sub_fixDur.Time = sub_fixDur.Time.astype(int)
# display(sub_fixDur)   


map_img = plt.imread('./map_grey.png')
# map_img = map_img.resize((450,500))
sns.set(context = "paper", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(12,9)})
# display(map_img.shape)
_, ax = plt.subplots(figsize=(20,20))

# cmap = sns.color_palette("Blues", as_cmap=True)
sub_fixDur.rename(columns={'Time':'Dwelling Time (msec)'}, inplace=True)

cmap = sns.cubehelix_palette(dark=0.2, light=0.8,rot=-.15, as_cmap=True)
ax.imshow(map_img, interpolation='nearest',aspect = 'auto',extent =[1,450,500,1], origin='upper')
ax = sns.scatterplot(x='y', y='x', hue='Dwelling Time (msec)', size='Dwelling Time (msec)',
                     sizes=(100, 1000), 
                     palette=cmap, 
                     data=sub_fixDur)


# plt.gca().invert_yaxis()
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_xlabel('')
plt.gca().set_ylabel('')
plt.legend(frameon =False)
plt.savefig(PLOTS_PATH+'/dwellingTime.png', quality=90, transparent=True)

In [ ]:
house_task_df = pd.read_csv(f'{CSV_PATH}/seahaven_VR_house_task_accuracy.csv')
display(house_task_df.info())
house_task_df[['PrimeNr','TargetNr','TargetNr_wrong']] = house_task_df[['PrimeNr','TargetNr','TargetNr_wrong']].astype(int, errors='ignore')
display(house_task_df.info())
house_task_subject_df = house_task_df.groupby(['Subject',''])
tmpdf = eye_df.join(rep_measurements, left_on='Subject', right_on='newPartNumber')
display(eye_df)
tmpdf = tmpdf.merge(house_task_df, left_on='Session3', right_on='Subject')
display(tmpdf)


# 9. Familiarity of houses based on MouseClicks (Map)

In [ ]:
mousedf = pd.read_csv(f'{MOUSE_DATA_PATH}/ClickStatsAccumulated3M.csv')
mousedf = pd.melt(mousedf, id_vars='House', value_vars = [ col for col in mousedf.columns if col !='House'], 
                  var_name='subject', value_name='clickNum' )
mousedf = mousedf.merge(house_df, left_on = 'House', right_on='HouseNr')
# display(mousedf)
clickdf = mousedf.groupby(['HouseNr','x', 'y' ]).agg({'clickNum':'mean'}).reset_index()
display(f'Mean Number of Clicks over houses and subjects: {clickdf.clickNum.mean()} (SD= {clickdf.clickNum.std()})')

clickdf.x = clickdf.x.astype(int) - 180
clickdf.y = clickdf.y.astype(int) - 535
clickdf.clickNum = clickdf.clickNum.astype(int)
map_img = plt.imread('./map_grey.png')
# map_img = map_img.resize((450,500))
sns.set(context = "paper", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(12,9)})
# display(map_img.shape)
_, ax = plt.subplots(figsize=(20,20))

# cmap = sns.color_palette("Blues", as_cmap=True)
clickdf.rename(columns={'clickNum':'Number of Clicks'}, inplace=True)

cmap = sns.cubehelix_palette(dark=0.2, light=0.8,rot=-.15, as_cmap=True)
ax.imshow(map_img, interpolation='nearest',aspect = 'auto',extent =[1,450,500,1], origin='upper')
ax = sns.scatterplot(x='y', y='x', hue='Number of Clicks', size='Number of Clicks',
                     sizes=(100, 1000), 
                     palette=cmap, 
                     data=clickdf)


# plt.gca().invert_yaxis()
plt.gca().spines['right'].set_visible(False)
plt.gca().spines['left'].set_visible(False)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['bottom'].set_visible(False)
plt.gca().set_xticklabels([''])
plt.gca().set_yticklabels([''])
plt.gca().set_xlabel('')
plt.gca().set_ylabel('')
plt.legend(frameon =False, loc='upper left')
plt.savefig(PLOTS_PATH+'/mouseclicks.png', quality=90, transparent=True)

In [ ]:
raise

# Scratch
### Getting valid subjects numbers from csv file 


In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_map_task_time_FRS.csv')
display(tmpdf.columns)
tmpdf = tmpdf.loc[:,['Absolute_3s', 'Absolute_Inf','Relative_3s',
       'Relative_Inf',  'Pointing_3s', 'Pointing_Inf', 'Egocentric', 'Allocentric',
       'Cardinal Direction']]
corr = tmpdf.corr(method='spearman')

sns.set(context = "paper", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(22,22)})

mask = np.zeros_like(corr)
display(mask)
mask[np.tril_indices_from(mask)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(corr,mask = mask, vmin=-1, vmax=1, square=True, center=0, 
                annot=True,linewidths=1, cmap="coolwarm", 
                fmt='.2f',annot_kws={"fontsize":18, 'color':'k'},
                cbar_kws={"shrink": .82})

plt.savefig(PLOTS_PATH+'/Map_task_time_FRS_correlation.png', quality=90)

In [ ]:
tmpdf = pd.read_csv(CSV_PATH + '/seahaven_VR_task_time_FRS.csv')
display(tmpdf.columns)
tmpdf = tmpdf.loc[:,['Absolute_3s', 'Absolute_Inf','Relative_3s',
       'Relative_Inf',  'Pointing_3s', 'Pointing_Inf', 'Egocentric', 'Allocentric',
       'Cardinal Direction']]
corr = tmpdf.corr(method='spearman')

sns.set(context = "paper", style="white", palette="Set1", font_scale=2.5, rc={'figure.figsize':(22,22)})

mask = np.zeros_like(corr)
display(mask)
mask[np.tril_indices_from(mask)] = True
with sns.axes_style("white"):
    ax = sns.heatmap(corr,mask = mask, vmin=-1, vmax=1, square=True, center=0, 
                annot=True,linewidths=1, cmap="coolwarm", 
                fmt='.2f',annot_kws={"fontsize":18, 'color':'k'},
                cbar_kws={"shrink": .82})

plt.savefig(PLOTS_PATH+'/VR_task_time_FRS_correlation.png', quality=90)
